In [1]:
#----------------------------------------------------
# Initial notebook for making the Illustris cosmic graph from galaxy catalogues
# Author: Christian Kragh Jespersen
# First created: 11/03/23 @KITP
# note: to make into routine later
#----------------------------------------------------

import h5py
import illustris_python as il
from torch_geometric.data import Data, DataLoader
import scipy.spatial as ss
import os, sys, torch, pickle
import os.path as osp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
## get illustris positions
tng_base_path = osp.expanduser("~/../../scratch/gpfs/cj1223/TNG50")
snapshot = 99
h = 0.7 # cosmology h, set to 1 if you want comoving Mpc

r_link = 4 #Mpc

cuts = {"minimum_log_stellar_mass": 9,
        "minimum_log_halo_mass": 8,
       "minimum_n_star_particles": 100}

undirected = True
periodic = False
use_loops = False

use_gal = True # True = use galaxy params to infer dark matter, False = use DM params to infer galaxy stuff

pad = 1 # how much padding to add around each subbox, so the separation in total will be 2 times this


if use_gal:
    use_cols = ['subhalo_x', 'subhalo_y', 'subhalo_z', 'subhalo_vx', 'subhalo_vy', 'subhalo_vz','subhalo_logstellarmass', 'subhalo_stellarhalfmassradius']
    y_cols = ['subhalo_loghalomass', 'subhalo_logvmax'] 
if not use_gal:
    use_cols = ['subhalo_x', 'subhalo_y', 'subhalo_z', 'subhalo_vx', 'subhalo_vy', 'subhalo_vz', 'subhalo_loghalomass', 'subhalo_logvmax'] 
    y_cols = ['subhalo_logstellarmass', 'subhalo_stellarhalfmassradius']

In [4]:
subhalo_fields = ["SubhaloPos", "SubhaloMassType"] #just an initial check here

subhalos = il.groupcat.loadSubhalos(tng_base_path, snapshot, fields=subhalo_fields) 

pos = subhalos["SubhaloPos"][:,:3]
min_box, max_box = np.rint(np.min(pos)), np.rint(np.max(pos))

box_size = max_box/(h*1e3) #/(h*1000), pos units are in kpc


subhalo_fields = [
        "SubhaloPos", "SubhaloMassType", "SubhaloLenType", "SubhaloHalfmassRadType", 
        "SubhaloVel", "SubhaloVmax", "SubhaloGrNr", "SubhaloFlag"
    ]
subhalos = il.groupcat.loadSubhalos(tng_base_path, snapshot, fields=subhalo_fields) 

halo_fields = ["Group_M_Crit200", "GroupFirstSub", "GroupPos", "GroupVel"]
halos = il.groupcat.loadHalos(tng_base_path, snapshot, fields=halo_fields)

subhalo_pos = subhalos["SubhaloPos"][:] / (h*1e3) #/(h*1000), pos units are in comoving kpc, so now in Mpc
subhalo_stellarmass = subhalos["SubhaloMassType"][:,4]
subhalo_halomass = subhalos["SubhaloMassType"][:,1]
subhalo_n_stellar_particles = subhalos["SubhaloLenType"][:,4]
subhalo_stellarhalfmassradius = subhalos["SubhaloHalfmassRadType"][:,4] #normalize?
subhalo_vel = subhalos["SubhaloVel"][:] #normalize?
subhalo_vmax = subhalos["SubhaloVmax"][:] #normalize?
subhalo_flag = subhalos["SubhaloFlag"][:]
halo_id = subhalos["SubhaloGrNr"][:]

halo_mass = halos["Group_M_Crit200"][:]
halo_primarysubhalo = halos["GroupFirstSub"][:]  # currently not used but might be good for magnitude gap
group_pos = halos["GroupPos"][:] / (h*1e3)
group_vel = halos["GroupVel"][:]  #normalize?

# get subhalos/galaxies      
subhalos = pd.DataFrame(
    np.column_stack([halo_id, subhalo_flag, np.arange(len(subhalo_stellarmass)), subhalo_pos, subhalo_vel, subhalo_n_stellar_particles, subhalo_stellarmass, subhalo_halomass, subhalo_stellarhalfmassradius, subhalo_vmax]), 
    columns=['halo_id', 'subhalo_flag', 'subhalo_id', 'subhalo_x', 'subhalo_y', 'subhalo_z', 'subhalo_vx', 'subhalo_vy', 'subhalo_vz', 'subhalo_n_stellar_particles', 'subhalo_stellarmass', 'subhalo_halomass', 'subhalo_stellarhalfmassradius', 'subhalo_vmax'],
)
subhalos = subhalos[subhalos["subhalo_flag"] != 0].copy()
subhalos['halo_id'] = subhalos['halo_id'].astype(int)
subhalos['subhalo_id'] = subhalos['subhalo_id'].astype(int)

subhalos.drop("subhalo_flag", axis=1, inplace=True)

# impose stellar mass and particle cuts
subhalos = subhalos[subhalos["subhalo_n_stellar_particles"] > cuts["minimum_n_star_particles"]].copy()
subhalos["subhalo_logstellarmass"] = np.log10(subhalos["subhalo_stellarmass"])+10

subhalos["subhalo_loghalomass"] = np.log10(subhalos["subhalo_halomass"])+10
subhalos["subhalo_logvmax"] = np.log10(subhalos["subhalo_vmax"])
subhalos["subhalo_logstellarhalfmassradius"] = np.log10(subhalos["subhalo_stellarhalfmassradius"])

subhalos = subhalos[subhalos["subhalo_loghalomass"] > cuts["minimum_log_halo_mass"]].copy()

subhalos = subhalos[subhalos["subhalo_logstellarmass"] > cuts["minimum_log_stellar_mass"]].copy()

subhalos.reset_index(drop = True)

data = []
for n in range(2):
    for g in range(2):
        for k in range(2):
            print(n,g,k)
            xlims = np.array([box_size/2*n+pad, box_size/2*(n+1)-pad])
            ylims = np.array([box_size/2*g+pad, box_size/2*(g+1)-pad])
            zlims = np.array([box_size/2*k+pad, box_size/2*(k+1)-pad])
            
            pos = np.vstack(subhalos[['subhalo_x', 'subhalo_y', 'subhalo_z']].to_numpy())
            
            xmask = np.logical_and(pos[:,0]>xlims[0],pos[:,0]<xlims[1])
            ymask = np.logical_and(pos[:,1]>ylims[0],pos[:,1]<ylims[1])
            zmask = np.logical_and(pos[:,2]>zlims[0],pos[:,2]<zlims[1])
            mask = np.logical_and(zmask, np.logical_and(xmask, ymask))
            
            df = subhalos.iloc[mask]
            df.reset_index(drop=True)

            # remove extraneous columns
            df.drop(["subhalo_n_stellar_particles", "subhalo_stellarmass", "subhalo_halomass"], axis=1, inplace=True)

            # set new zero point
            
            df[['subhalo_x', 'subhalo_y', 'subhalo_z']] = df[['subhalo_x', 'subhalo_y', 'subhalo_z']] - np.array([box_size/2*n+pad, box_size/2*g+pad, box_size/2*k+pad])
            
            #make positions for clustering

            pos = np.vstack(df[['subhalo_x', 'subhalo_y', 'subhalo_z']].to_numpy())

            kd_tree = ss.KDTree(pos, leafsize=25, boxsize=box_size)
            edge_index = kd_tree.query_pairs(r=r_link, output_type="ndarray")
            
            # normalize positions
            
            df[['subhalo_x', 'subhalo_y', 'subhalo_z']] = df[['subhalo_x', 'subhalo_y', 'subhalo_z']]/(box_size/2)
            
            if undirected:
            # Add reverse pairs
                reversepairs = np.zeros((edge_index.shape[0],2))
                for i, pair in enumerate(edge_index):
                    reversepairs[i] = np.array([pair[1], pair[0]])
                edge_index = np.append(edge_index, reversepairs, 0)

                edge_index = edge_index.astype(int)

                # Write in pytorch-geometric format
                edge_index = edge_index.reshape((2,-1))
                num_pairs = edge_index.shape[1]

            row, col = edge_index

            diff = pos[row]-pos[col]
            dist = np.linalg.norm(diff, axis=1)

            use_gal = True

            if periodic:
                # Take into account periodic boundary conditions, correcting the distances
                for i, pos_i in enumerate(diff):
                    for j, coord in enumerate(pos_i):
                        if coord > r_link:
                            diff[i,j] -= box_size  # Boxsize normalize to 1
                        elif -coord > r_link:
                            diff[i,j] += box_size  # Boxsize normalize to 1

            centroid = np.mean(pos,axis=0) # define arbitrary coordinate, invarinat to translation/rotation shifts, but not stretches
            # centroid+=1.2

            unitrow = (pos[row]-centroid)/np.linalg.norm((pos[row]-centroid), axis=1).reshape(-1,1)
            unitcol = (pos[col]-centroid)/np.linalg.norm((pos[col]-centroid), axis=1).reshape(-1,1)
            unitdiff = diff/dist.reshape(-1,1)
            # Dot products between unit vectors
            cos1 = np.array([np.dot(unitrow[i,:].T,unitcol[i,:]) for i in range(num_pairs)])
            cos2 = np.array([np.dot(unitrow[i,:].T,unitdiff[i,:]) for i in range(num_pairs)])

            edge_attr = np.concatenate([dist.reshape(-1,1), cos1.reshape(-1,1), cos2.reshape(-1,1)], axis=1)

            if use_loops:
                loops = np.zeros((2,pos.shape[0]),dtype=int)
                atrloops = np.zeros((pos.shape[0],3))
                for i, posit in enumerate(pos):
                    loops[0,i], loops[1,i] = i, i
                    atrloops[i,0], atrloops[i,1], atrloops[i,2] = 0., 1., 0.
                edge_index = np.append(edge_index, loops, 1)
                edge_attr = np.append(edge_attr, atrloops, 0)
            edge_index = edge_index.astype(int)

            x =  torch.tensor(np.vstack(df[use_cols].to_numpy()), dtype=torch.float)
            y =  torch.tensor(np.vstack(df[y_cols].to_numpy()), dtype=torch.float)
            edge_index = torch.tensor(edge_index, dtype=torch.long)
            edge_attr=torch.tensor(edge_attr, dtype=torch.float)

            data.append(Data(x = x, y= y, edge_index = edge_index, edge_attr = edge_attr))
            
            data_path = osp.join(tng_base_path, 'cosmic_graphs', f'split_{len(data)}_link_{int(r_link)}_pad{int(pad)}_gal{int(use_gal)}.pkl')

            if not osp.isdir(osp.join(tng_base_path, 'cosmic_graphs')):
                os.mkdir(osp.join(tng_base_path, 'cosmic_graphs'))

            with open(data_path, 'wb') as handle:
                pickle.dump(data, handle)

/home/cj1223/.conda/envs/torch-env/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/tmp/ipykernel_127319/3650068997.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(["subhalo_n_stellar_particles", "subhalo_stellarmass", "subhalo_halomass"], axis=1, inplace=True)
/tmp/ipykernel_127319/3650068997.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['subhalo_x', 'subhalo_y', 'subhalo_z']] = df[['subhalo_x', 'subhalo_y', 'subhalo_z']] - 

0 0 0
0 0 1
0 1 0
0 1 1
1 0 0
1 0 1
1 1 0
1 1 1


/tmp/ipykernel_127319/3650068997.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(["subhalo_n_stellar_particles", "subhalo_stellarmass", "subhalo_halomass"], axis=1, inplace=True)
/tmp/ipykernel_127319/3650068997.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['subhalo_x', 'subhalo_y', 'subhalo_z']] = df[['subhalo_x', 'subhalo_y', 'subhalo_z']] - np.array([box_size/2*n+pad, box_size/2*g+pad, box_size/2*k+pad])
/tmp/ipykernel_127319/3650068997.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [5]:
data

[Data(x=[136, 8], edge_index=[2, 1440], edge_attr=[1440, 3], y=[136, 2]),
 Data(x=[187, 8], edge_index=[2, 2354], edge_attr=[2354, 3], y=[187, 2]),
 Data(x=[185, 8], edge_index=[2, 2560], edge_attr=[2560, 3], y=[185, 2]),
 Data(x=[438, 8], edge_index=[2, 23388], edge_attr=[23388, 3], y=[438, 2]),
 Data(x=[411, 8], edge_index=[2, 17430], edge_attr=[17430, 3], y=[411, 2]),
 Data(x=[225, 8], edge_index=[2, 3320], edge_attr=[3320, 3], y=[225, 2]),
 Data(x=[264, 8], edge_index=[2, 3520], edge_attr=[3520, 3], y=[264, 2]),
 Data(x=[154, 8], edge_index=[2, 1618], edge_attr=[1618, 3], y=[154, 2])]

In [6]:
data[7].x

tensor([[ 7.4054e-01,  3.2203e-01,  7.9618e-01,  ...,  4.5642e+01,
          1.1350e+01,  1.1738e+01],
        [ 7.4090e-01,  3.2256e-01,  7.9545e-01,  ...,  1.9132e+02,
          1.0687e+01,  7.6678e-01],
        [ 7.3966e-01,  3.2176e-01,  7.9621e-01,  ...,  2.4810e+02,
          9.7094e+00,  5.0422e-01],
        ...,
        [ 1.7889e-01,  6.3071e-03,  1.1617e-01,  ..., -1.9223e+02,
          9.0914e+00,  1.1068e+00],
        [ 3.7151e-01,  4.1904e-01,  1.3596e-01,  ...,  5.2544e+01,
          9.0099e+00,  1.5927e+00],
        [ 2.0572e-01,  4.8980e-01,  3.0928e-01,  ..., -9.3872e+01,
          9.0909e+00,  2.4965e+00]])

In [7]:
import torch_geometric.utils as tg_utils
nx = tg_utils.to_networkx(data[0])

In [8]:
def visualize_graph(data, idx = 0, draw_edges = True, sizes=0.1, projection="3d", edge_index=None, fontsize = 11):

    fig = plt.figure(figsize=(4, 4))

    if projection=="3d":
        ax = fig.add_subplot(projection ="3d")
        pos = data.x[:,:3]
    elif projection=="2d":
        ax = fig.add_subplot()
        pos = data.x[:,:2]

    # Draw lines for each edge
    if data.edge_index is not None and draw_edges:
        for (src, dst) in data.edge_index.t().tolist():

            src = pos[src].tolist()
            dst = pos[dst].tolist()
    #         print(src, dst
            if projection=="3d":
                ax.plot([src[0], dst[0]], [src[1], dst[1]], zs=[src[2], dst[2]], linewidth=sizes/r_link, color='black')
            elif projection=="2d":
                ax.plot([src[0], dst[0]], [src[1], dst[1]], linewidth=sizes/r_link, color='black')

    # Plot nodes
    if projection=="3d":
        ax.scatter(pos[:, 0], pos[:, 1], pos[:, 2], s=sizes, zorder=1000, alpha=0.5)
    elif projection=="2d":
        ax.scatter(pos[:, 0], pos[:, 1], s=sizes, zorder=1000, alpha=0.5)

    ax.xaxis.set_tick_params(labelsize=fontsize)
    ax.yaxis.set_tick_params(labelsize=fontsize)
    ax.zaxis.set_tick_params(labelsize=fontsize)

    fig.savefig(f"figs/graph_for_John_{idx}.png", bbox_inches='tight', dpi=300)
    plt.close(fig)


In [9]:
for i in range(len(data)):
    visualize_graph(data[i], idx = i)